In [190]:
import pandas as pd
import numpy as np
from pyproj import Transformer
import warnings
 
# suppress warnings
warnings.filterwarnings('ignore')

In [191]:
# # make copies of ABR, AVA, NEO, and VEG
# sources = ['ABR', 'AVA', 'NEO', 'VEG']
# filetype = "parent"

# # Read all the files
# for source in sources:
#     df = pd.read_csv(f"/mnt/poseidon/remotesensing/arctic/data/training/Test_06/fcover/{source}_fcover_{filetype}_01.csv", index_col=0)
#     df.to_csv(f"/mnt/poseidon/remotesensing/arctic/data/training/Test_06/fcover/{source}_fcover_{filetype}_03.csv")

In [192]:
# add back plot_radius_m
orig_ancillary = pd.read_csv("/mnt/poseidon/remotesensing/arctic/alaska_pft_fcover_harmonization/data/plot_data/nga/input_data/nga_ancillary.csv")
orig_ancillary

,TID,Site Code,plot_radius_m,date,latitude,longitude,year,source,dataset
0,kk,KG_AS1_VgComp,2.50,20160723.0,65.164850,-164.823195,2016.0,SP,ngee_arctic_kougarok
1,ll,KG_AS2_VgComp,2.50,20160723.0,65.158620,-164.822328,2016.0,SP,ngee_arctic_kougarok
2,mm,KG_AS3_VgComp,2.50,20160723.0,65.161775,-164.831645,2016.0,SP,ngee_arctic_kougarok
3,nn,KG_AS4_VgComp,2.50,20160723.0,65.164293,-164.823757,2016.0,SP,ngee_arctic_kougarok
4,oo,KG_AS5_VgComp,2.50,20160723.0,65.157525,-164.825048,2016.0,SP,ngee_arctic_kougarok
...,...,...,...,...,...,...,...,...,...
96,ssss,CL_WAC-5-1_Vg_Comp,1.25,20180801.0,64.859265,-163.690300,2018.0,SP,ngee_arctic_council
97,tttt,CL_WAC2_Vg_Comp,1.25,20180801.0,64.860260,-163.697757,2018.0,SP,ngee_arctic_council
98,uuuu,CL_WAC3_Vg_Comp,1.25,20180801.0,64.856472,-163.689893,2018.0,SP,ngee_arctic_council
99,vvvv,CL_WAC4_Vg_Comp,1.25,20180801.0,64.859385,-163.703835,2018.0,SP,ngee_arctic_council


In [193]:
# now re-do SP/NGA
import pandas as pd
import numpy as np
from pyproj import Transformer
import warnings
from ast import literal_eval
 
# suppress warnings
warnings.filterwarnings('ignore')

# parameters
src = 'nga'
dst = 'SP'
vers = '03'
wdir = '../../data/training/Test_06/fcover'
dist_thres = 55          # meters
small_area_thr = 314     # m-2 threshold for small plots
small_rad_thr = 10

# --- 0) Set export naming schema ---
schema = {
    'surveyYear':'year',
    'latitudeY':'latitude',
    'longitudeX':'longitude',
    'dataSource':'source',
    'dataSubsource':'subsource',
    'surveyMethod':'field sampling method',
    'fcoverScale':'cover measurement',
    'baregroundCover':'bare ground top cover (%)',
    'bryophyteCover':'bryophyte total cover (%)',
    'deciduousShrubCover':'deciduous shrub total cover (%)',
    'deciduousTreeCover':'deciduous tree total cover (%)',
    'evergreenShrubCover':'evergreen shrub total cover (%)',
    'evergreenTreeCover':'evergreen tree total cover (%)',
    'forbCover':'forb total cover (%)',
    'graminoidCover':'graminoid total cover (%)',
    'lichenCover':'lichen total cover (%)',
    'litterCover':'litter total cover (%)',
    'nonvascularSumCover':'non-vascular total cover (%)',
    'waterCover':'water top cover (%)'}


# --- 1) Read & export child_data ---
# read cover and plot info
override_dtype = {'plotVisit': str}
cover = pd.read_csv(f'../data/plot_data/{src}/output_data/{src}_standard_pft_fcover.csv', dtype=override_dtype)
cover

,plotVisit,deciduousShrubCover,deciduousTreeCover,evergreenShrubCover,evergreenTreeCover,forbCover,graminoidCover,nonvascularSumCover,bryophyteCover,lichenCover,litterCover,baregroundCover,waterCover,otherCover
0,u8efd86fb7,99.0,NaN,6.0,NaN,53.5,14.0,7.0,7.0,NaN,45.0,0.0,0.0,NaN
1,u9a900f538,83.0,NaN,25.0,NaN,8.0,6.0,30.0,30.0,NaN,25.0,0.0,0.0,NaN
2,ud3fc13dc1,18.0,NaN,3.0,NaN,3.0,41.0,55.0,55.0,NaN,0.0,0.0,4.0,NaN
3,u3918373cf,19.0,NaN,3.0,NaN,2.0,70.0,42.0,42.0,NaN,0.0,1.0,0.0,NaN
4,u4dc7c9ec4,88.0,NaN,17.0,NaN,55.0,8.0,14.0,14.0,NaN,15.0,0.0,0.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
92,ubed4eb698,96.0,NaN,55.0,NaN,0.5,69.0,9.0,6.5,2.5,0.0,4.0,0.0,NaN
93,u8c1017982,59.0,NaN,27.0,NaN,3.0,30.0,14.5,10.5,4.0,0.0,0.0,0.0,NaN
94,u07dcd3715,15.0,NaN,43.0,NaN,5.0,88.0,8.5,7.5,1.0,0.0,0.0,0.0,NaN
95,u6d3236ec3,82.0,NaN,50.0,NaN,10.0,26.0,19.5,15.0,4.5,0.0,0.0,0.0,NaN


In [194]:
info = pd.read_csv(f'../data/plot_data/{src}/output_data/{src}_plot_info.csv', dtype=override_dtype)
info = info[[
    'plotVisit','plotName', 'surveyYear',
    'latitudeY','longitudeX','dataSource','dataSubsource',
    'plotArea','plotShape','surveyMethod','fcoverScale', 
]]

# add plot_radius_m
info = info.merge(orig_ancillary[['Site Code', 'plot_radius_m']], how='left', left_on='plotName', right_on='Site Code')
info = info.drop(columns=['plotShape', 'plotArea', 'Site Code'])
info

,plotVisit,plotName,surveyYear,latitudeY,longitudeX,dataSource,dataSubsource,surveyMethod,fcoverScale,plot_radius_m
0,u8efd86fb7,TL_MS8_VgComp,2017,64.727897,-165.943768,NGA,NGEE-Arctic Amy Breen (pub 2020),simple plot,percent,1.25
1,u9a900f538,TL_WBT8_VgComp,2017,64.728535,-165.943505,NGA,NGEE-Arctic Amy Breen (pub 2020),simple plot,percent,1.25
2,ud3fc13dc1,TL_WNAC5_VgComp,2017,64.729370,-165.944615,NGA,NGEE-Arctic Amy Breen (pub 2020),simple plot,percent,1.25
3,u3918373cf,TL_WNAC6_VgComp,2017,64.729770,-165.944855,NGA,NGEE-Arctic Amy Breen (pub 2020),simple plot,percent,1.25
4,u4dc7c9ec4,TL_MS6_VgComp,2017,64.731115,-165.946545,NGA,NGEE-Arctic Amy Breen (pub 2020),simple plot,percent,1.25
...,...,...,...,...,...,...,...,...,...,...
92,ubed4eb698,KG_ASV2_VgComp,2016,65.165445,-164.821015,NGA,NGEE-Arctic Amy Breen (pub 2020),simple plot,percent,1.25
93,u8c1017982,KG_ASV5_VgComp,2016,65.166040,-164.818240,NGA,NGEE-Arctic Amy Breen (pub 2020),simple plot,percent,1.25
94,u07dcd3715,KG_TT3_VgComp,2016,65.166275,-164.818973,NGA,NGEE-Arctic Amy Breen (pub 2020),simple plot,percent,1.25
95,u6d3236ec3,KG_ASV1_VgComp,2016,65.166678,-164.815835,NGA,NGEE-Arctic Amy Breen (pub 2020),simple plot,percent,1.25


In [195]:
# replace plotVisit names with the ones used in test 06, training data 01
merged_df = cover.merge(info[['plotVisit', 'plotName']], on='plotVisit', how='left')
cover['plotVisit'] = merged_df['plotName']
cover

,plotVisit,deciduousShrubCover,deciduousTreeCover,evergreenShrubCover,evergreenTreeCover,forbCover,graminoidCover,nonvascularSumCover,bryophyteCover,lichenCover,litterCover,baregroundCover,waterCover,otherCover
0,TL_MS8_VgComp,99.0,NaN,6.0,NaN,53.5,14.0,7.0,7.0,NaN,45.0,0.0,0.0,NaN
1,TL_WBT8_VgComp,83.0,NaN,25.0,NaN,8.0,6.0,30.0,30.0,NaN,25.0,0.0,0.0,NaN
2,TL_WNAC5_VgComp,18.0,NaN,3.0,NaN,3.0,41.0,55.0,55.0,NaN,0.0,0.0,4.0,NaN
3,TL_WNAC6_VgComp,19.0,NaN,3.0,NaN,2.0,70.0,42.0,42.0,NaN,0.0,1.0,0.0,NaN
4,TL_MS6_VgComp,88.0,NaN,17.0,NaN,55.0,8.0,14.0,14.0,NaN,15.0,0.0,0.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
92,KG_ASV2_VgComp,96.0,NaN,55.0,NaN,0.5,69.0,9.0,6.5,2.5,0.0,4.0,0.0,NaN
93,KG_ASV5_VgComp,59.0,NaN,27.0,NaN,3.0,30.0,14.5,10.5,4.0,0.0,0.0,0.0,NaN
94,KG_TT3_VgComp,15.0,NaN,43.0,NaN,5.0,88.0,8.5,7.5,1.0,0.0,0.0,0.0,NaN
95,KG_ASV1_VgComp,82.0,NaN,50.0,NaN,10.0,26.0,19.5,15.0,4.5,0.0,0.0,0.0,NaN


In [196]:
# merge and filter by year ≥ 2010
orig_child = pd.read_csv(f'{wdir}/{dst}_fcover_child_01.csv')
orig_child

,Site Code,bare ground top cover (%),bryophyte total cover (%),deciduous shrub total cover (%),deciduous tree total cover (%),evergreen shrub total cover (%),evergreen tree total cover (%),forb total cover (%),graminoid total cover (%),lichen total cover (%),litter total cover (%),non-vascular total cover (%),water top cover (%),latitude,longitude,plot_radius_m,source,year,field sampling method,cover measurement
0,CL_EL1_Vg_Comp,1,9.0,15.0,0,51.0,0,0.0,3.0,48.0,2,57.0,0,64.845250,-163.782758,1.25,SP,2018,quadrat,percent
1,CL_EL2_Vg_Comp,3,5.0,6.0,0,42.0,0,2.0,4.0,81.0,2,86.0,0,64.845515,-163.784122,1.25,SP,2018,quadrat,percent
2,CL_PMRB1_Vg_Comp,0,66.0,3.0,0,16.0,0,1.0,21.0,1.0,5,67.0,0,64.859610,-163.691640,1.25,SP,2018,quadrat,percent
3,CL_PMRB2_Vg_Comp,0,28.0,1.0,0,6.0,0,2.0,31.0,2.0,8,30.0,0,64.859995,-163.687948,1.25,SP,2018,quadrat,percent
4,CL_PMRB3_Vg_Comp,2,61.0,11.0,0,14.0,0,1.0,17.0,0.0,10,61.0,0,64.856385,-163.692985,1.25,SP,2018,quadrat,percent
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
93,TL_WNAC3_VgComp,0,12.0,0.0,0,0.0,0,0.0,0.0,0.0,0,12.0,0,64.743175,-165.963990,1.25,SP,2017,quadrat,percent
94,TL_WNAC4_VgComp,0,37.0,0.0,0,0.0,0,0.0,0.0,0.0,0,37.0,0,64.739357,-165.971090,1.25,SP,2017,quadrat,percent
95,TL_WNAC5_VgComp,0,55.0,0.0,0,0.0,0,0.0,0.0,0.0,0,55.0,0,64.729370,-165.944615,1.25,SP,2017,quadrat,percent
96,TL_WNAC6_VgComp,0,42.0,0.0,0,0.0,0,0.0,0.0,0.0,0,42.0,0,64.729770,-165.944855,1.25,SP,2017,quadrat,percent


In [197]:
child = cover.merge(info, left_on='plotVisit', right_on='plotName', how='left', suffixes=('','_x'))
child = child.drop(columns=[col for col in child.columns.to_list() if col.endswith('_x')])
child = child[child['surveyYear'] >= 2010].reset_index(drop=True)
child = child[child['plotVisit'].isin(orig_child['Site Code'])].reset_index(drop=True)
child = child.sort_values(by='plotVisit').reset_index(drop=True)
child

,plotVisit,deciduousShrubCover,deciduousTreeCover,evergreenShrubCover,evergreenTreeCover,forbCover,graminoidCover,nonvascularSumCover,bryophyteCover,lichenCover,...,otherCover,plotName,surveyYear,latitudeY,longitudeX,dataSource,dataSubsource,surveyMethod,fcoverScale,plot_radius_m
0,CL_AS2-1_Vg_Comp,15.0,NaN,51.0,0.0,0.0,3.0,57.0,9.0,48.0,...,NaN,CL_AS2-1_Vg_Comp,2018,64.847662,-163.756327,NGA,NGEE-Arctic Amy Breen (pub 2020),simple plot,percent,2.50
1,CL_AS3_Vg_Comp,6.0,NaN,42.0,0.0,2.0,4.0,86.0,5.0,81.0,...,NaN,CL_AS3_Vg_Comp,2018,64.845465,-163.759130,NGA,NGEE-Arctic Amy Breen (pub 2020),simple plot,percent,2.50
2,CL_AS4_Vg_Comp,0.0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,CL_AS4_Vg_Comp,2018,64.853408,-163.750220,NGA,NGEE-Arctic Amy Breen (pub 2020),simple plot,percent,2.50
3,CL_AS5_Vg_Comp,0.0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,CL_AS5_Vg_Comp,2018,64.845692,-163.772085,NGA,NGEE-Arctic Amy Breen (pub 2020),simple plot,percent,2.50
4,CL_AS8-2_Vg_Comp,0.0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,CL_AS8-2_Vg_Comp,2018,64.836740,-163.730525,NGA,NGEE-Arctic Amy Breen (pub 2020),simple plot,percent,2.50
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
92,TL_WNAC3_VgComp,9.0,NaN,4.0,NaN,4.0,77.0,12.0,12.0,NaN,...,NaN,TL_WNAC3_VgComp,2017,64.743175,-165.963990,NGA,NGEE-Arctic Amy Breen (pub 2020),simple plot,percent,1.25
93,TL_WNAC4_VgComp,23.0,NaN,23.0,NaN,0.0,19.0,37.0,37.0,NaN,...,NaN,TL_WNAC4_VgComp,2017,64.739357,-165.971090,NGA,NGEE-Arctic Amy Breen (pub 2020),simple plot,percent,1.25
94,TL_WNAC5_VgComp,18.0,NaN,3.0,NaN,3.0,41.0,55.0,55.0,NaN,...,NaN,TL_WNAC5_VgComp,2017,64.729370,-165.944615,NGA,NGEE-Arctic Amy Breen (pub 2020),simple plot,percent,1.25
95,TL_WNAC6_VgComp,19.0,NaN,3.0,NaN,2.0,70.0,42.0,42.0,NaN,...,NaN,TL_WNAC6_VgComp,2017,64.729770,-165.944855,NGA,NGEE-Arctic Amy Breen (pub 2020),simple plot,percent,1.25


In [198]:
# export prep
child_out = child.copy()
child_out = child_out.rename(columns=schema)
child_out.index = child_out['plotVisit']
child_out.index.name = 'Site Code'
child_out = child_out.drop(columns=['plotVisit', 'plotName', 'otherCover', 'subsource'])
child_out.columns

Index(['deciduous shrub total cover (%)', 'deciduous tree total cover (%)',
       'evergreen shrub total cover (%)', 'evergreen tree total cover (%)',
       'forb total cover (%)', 'graminoid total cover (%)',
       'non-vascular total cover (%)', 'bryophyte total cover (%)',
       'lichen total cover (%)', 'litter total cover (%)',
       'bare ground top cover (%)', 'water top cover (%)', 'year', 'latitude',
       'longitude', 'source', 'field sampling method', 'cover measurement',
       'plot_radius_m'],
      dtype='object')

In [199]:
# sort to match original 01
desired_order = orig_child['Site Code'].tolist()
child_out = child_out.reindex(desired_order)
child_out

,deciduous shrub total cover (%),deciduous tree total cover (%),evergreen shrub total cover (%),evergreen tree total cover (%),forb total cover (%),graminoid total cover (%),non-vascular total cover (%),bryophyte total cover (%),lichen total cover (%),litter total cover (%),bare ground top cover (%),water top cover (%),year,latitude,longitude,source,field sampling method,cover measurement,plot_radius_m
Site Code,,,,,,,,,,,,,,,,,,,
CL_EL1_Vg_Comp,15.0,NaN,2.0,0.0,24.0,56.0,93.0,93.0,0.0,15.0,0.0,0.0,2018.0,64.845250,-163.782758,NGA,simple plot,percent,1.25
CL_EL2_Vg_Comp,17.0,NaN,14.0,0.0,0.0,37.0,64.0,61.0,3.0,12.0,0.0,4.0,2018.0,64.845515,-163.784122,NGA,simple plot,percent,1.25
CL_PMRB1_Vg_Comp,37.0,NaN,22.0,0.0,1.0,3.0,125.0,3.0,122.0,3.0,0.0,0.0,2018.0,64.859610,-163.691640,NGA,simple plot,percent,1.25
CL_PMRB2_Vg_Comp,26.0,NaN,16.0,0.0,0.0,2.0,105.0,6.0,99.0,5.0,0.0,0.0,2018.0,64.859995,-163.687948,NGA,simple plot,percent,1.25
CL_PMRB3_Vg_Comp,45.0,NaN,34.0,0.0,2.0,0.0,93.0,3.0,90.0,2.0,0.0,0.0,2018.0,64.856385,-163.692985,NGA,simple plot,percent,1.25
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
TL_WNAC3_VgComp,9.0,NaN,4.0,NaN,4.0,77.0,12.0,12.0,NaN,0.0,1.0,5.0,2017.0,64.743175,-165.963990,NGA,simple plot,percent,1.25
TL_WNAC4_VgComp,23.0,NaN,23.0,NaN,0.0,19.0,37.0,37.0,NaN,0.0,0.0,0.0,2017.0,64.739357,-165.971090,NGA,simple plot,percent,1.25
TL_WNAC5_VgComp,18.0,NaN,3.0,NaN,3.0,41.0,55.0,55.0,NaN,0.0,0.0,4.0,2017.0,64.729370,-165.944615,NGA,simple plot,percent,1.25


In [200]:
sorted_cols = [
    'bare ground top cover (%)', 'bryophyte total cover (%)',
    'deciduous shrub total cover (%)', 'deciduous tree total cover (%)',
    'evergreen shrub total cover (%)', 'evergreen tree total cover (%)',
    'forb total cover (%)', 'graminoid total cover (%)',
    'lichen total cover (%)', 'litter total cover (%)',
    'non-vascular total cover (%)', 'water top cover (%)',
    'latitude', 'longitude', 'plot_radius_m', 'source', 'year', 
    'field sampling method', 'cover measurement']
child_out = child_out[sorted_cols]
child_out['year'] = child_out['year'].astype('Int64')
child_out['source'] = 'SP'
child_out['field sampling method'] = 'quadrat'
child_out.to_csv(f'{wdir}/{dst}_fcover_child_{vers}.csv')

In [201]:
# project to UTM once, and stash coords on every row
child_out = child_out.reset_index()
transformer = Transformer.from_crs("EPSG:4326", "EPSG:32606", always_xy=True)
xy = np.array([transformer.transform(lon, lat)
               for lon, lat in zip(child_out.longitude, child_out.latitude)])
child_out['x'], child_out['y'] = xy[:,0], xy[:,1]

In [202]:
# --- 2) cluster small plots by distance ---
def group_by_distance(points, threshold):
    labels = np.full(len(points), -1, dtype=int)
    clusters = {}
    cid = 0
    for i, pt in enumerate(points):
        for k, members in clusters.items():
            if np.any(np.linalg.norm(np.vstack(members) - pt, axis=1) <= threshold):
                clusters[k].append(pt)
                labels[i] = k
                break
        else:
            clusters[cid] = [pt]
            labels[i] = cid
            cid += 1
    return labels

# --- recalc area & shape with list comprehensions instead of apply() ---
# helper for max-pairwise distance (unchanged)
def max_pairwise_dist(xs, ys):
    pts = np.column_stack((xs, ys))
    if pts.shape[0] < 2:
        return 0.0
    d = np.linalg.norm(pts[:, None, :] - pts[None, :, :], axis=2)
    return d.max()

# helper for computing each parent’s area
def compute_parent_area(xs, ys, orig_list):
    if len(xs) < 2:
        return orig_list[0]
    dmax = max_pairwise_dist(xs, ys)
    return np.pi * (dmax / 2) ** 2

# 1) Give every row a default group_id = its own plotVisit
child_out['group_id'] = child_out['Site Code']
child_out

,Site Code,bare ground top cover (%),bryophyte total cover (%),deciduous shrub total cover (%),deciduous tree total cover (%),evergreen shrub total cover (%),evergreen tree total cover (%),forb total cover (%),graminoid total cover (%),lichen total cover (%),...,latitude,longitude,plot_radius_m,source,year,field sampling method,cover measurement,x,y,group_id
0,CL_EL1_Vg_Comp,0.0,93.0,15.0,NaN,2.0,0.0,24.0,56.0,0.0,...,64.845250,-163.782758,1.25,SP,2018,quadrat,percent,-288638.520067,7.296783e+06,CL_EL1_Vg_Comp
1,CL_EL2_Vg_Comp,0.0,61.0,17.0,NaN,14.0,0.0,0.0,37.0,3.0,...,64.845515,-163.784122,1.25,SP,2018,quadrat,percent,-288693.614090,7.296829e+06,CL_EL2_Vg_Comp
2,CL_PMRB1_Vg_Comp,0.0,3.0,37.0,NaN,22.0,0.0,1.0,3.0,122.0,...,64.859610,-163.691640,1.25,SP,2018,quadrat,percent,-284014.587205,7.297196e+06,CL_PMRB1_Vg_Comp
3,CL_PMRB2_Vg_Comp,0.0,6.0,26.0,NaN,16.0,0.0,0.0,2.0,99.0,...,64.859995,-163.687948,1.25,SP,2018,quadrat,percent,-283833.064238,7.297191e+06,CL_PMRB2_Vg_Comp
4,CL_PMRB3_Vg_Comp,0.0,3.0,45.0,NaN,34.0,0.0,2.0,0.0,90.0,...,64.856385,-163.692985,1.25,SP,2018,quadrat,percent,-284171.455611,7.296863e+06,CL_PMRB3_Vg_Comp
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
93,TL_WNAC3_VgComp,1.0,12.0,9.0,NaN,4.0,NaN,4.0,77.0,NaN,...,64.743175,-165.963990,1.25,SP,2017,quadrat,percent,-392211.045243,7.315054e+06,TL_WNAC3_VgComp
94,TL_WNAC4_VgComp,0.0,37.0,23.0,NaN,23.0,NaN,0.0,19.0,NaN,...,64.739357,-165.971090,1.25,SP,2017,quadrat,percent,-392664.527176,7.314745e+06,TL_WNAC4_VgComp
95,TL_WNAC5_VgComp,0.0,55.0,18.0,NaN,3.0,NaN,3.0,41.0,NaN,...,64.729370,-165.944615,1.25,SP,2017,quadrat,percent,-391782.104376,7.313294e+06,TL_WNAC5_VgComp
96,TL_WNAC6_VgComp,1.0,42.0,19.0,NaN,3.0,NaN,2.0,70.0,NaN,...,64.729770,-165.944855,1.25,SP,2017,quadrat,percent,-391779.784721,7.313340e+06,TL_WNAC6_VgComp


In [203]:
# 2) Identify the small ones, cluster *only* those, and override group_id there
small_mask = child_out['plot_radius_m'] < small_rad_thr
labels = np.array([], dtype=int)
if small_mask.any():
    coords = child_out.loc[small_mask, ['x','y']].to_numpy()
    labels = group_by_distance(coords, dist_thres)
    child_out.loc[small_mask, 'group_id'] = labels.astype(str)

# 3) Did *any* small cluster actually merge >1 point?
merged = pd.Series(labels).value_counts().gt(1).any() if small_mask.any() else False

# --- build parents DF ---
parents = child_out.set_index('Site Code')
# aggregated to parent flag
if small_mask.any():
    group_counts = pd.Series(labels).value_counts()
else:
    group_counts = pd.Series(dtype=int)

# create aggregate column
parents['aggregated to parent'] = parents['group_id'].map(
    lambda g: 'yes' if group_counts.get(int(g) if str(g).isdigit() else g, 0) > 1 else 'no'
)

# create set of child site codes
parents['child site codes'] = parents['group_id'].map(
    lambda g: set(child_out.loc[child_out['group_id']==g, 'Site Code'])
)
parents

,bare ground top cover (%),bryophyte total cover (%),deciduous shrub total cover (%),deciduous tree total cover (%),evergreen shrub total cover (%),evergreen tree total cover (%),forb total cover (%),graminoid total cover (%),lichen total cover (%),litter total cover (%),...,plot_radius_m,source,year,field sampling method,cover measurement,x,y,group_id,aggregated to parent,child site codes
Site Code,,,,,,,,,,,,,,,,,,,,,
CL_EL1_Vg_Comp,0.0,93.0,15.0,NaN,2.0,0.0,24.0,56.0,0.0,15.0,...,1.25,SP,2018,quadrat,percent,-288638.520067,7.296783e+06,0,no,{CL_EL1_Vg_Comp}
CL_EL2_Vg_Comp,0.0,61.0,17.0,NaN,14.0,0.0,0.0,37.0,3.0,12.0,...,1.25,SP,2018,quadrat,percent,-288693.614090,7.296829e+06,1,no,{CL_EL2_Vg_Comp}
CL_PMRB1_Vg_Comp,0.0,3.0,37.0,NaN,22.0,0.0,1.0,3.0,122.0,3.0,...,1.25,SP,2018,quadrat,percent,-284014.587205,7.297196e+06,2,no,{CL_PMRB1_Vg_Comp}
CL_PMRB2_Vg_Comp,0.0,6.0,26.0,NaN,16.0,0.0,0.0,2.0,99.0,5.0,...,1.25,SP,2018,quadrat,percent,-283833.064238,7.297191e+06,3,no,{CL_PMRB2_Vg_Comp}
CL_PMRB3_Vg_Comp,0.0,3.0,45.0,NaN,34.0,0.0,2.0,0.0,90.0,2.0,...,1.25,SP,2018,quadrat,percent,-284171.455611,7.296863e+06,4,no,{CL_PMRB3_Vg_Comp}
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
TL_WNAC3_VgComp,1.0,12.0,9.0,NaN,4.0,NaN,4.0,77.0,NaN,0.0,...,1.25,SP,2017,quadrat,percent,-392211.045243,7.315054e+06,82,no,{TL_WNAC3_VgComp}
TL_WNAC4_VgComp,0.0,37.0,23.0,NaN,23.0,NaN,0.0,19.0,NaN,0.0,...,1.25,SP,2017,quadrat,percent,-392664.527176,7.314745e+06,69,yes,"{TL_MM4_VgComp, TL_WNAC4_VgComp}"
TL_WNAC5_VgComp,0.0,55.0,18.0,NaN,3.0,NaN,3.0,41.0,NaN,0.0,...,1.25,SP,2017,quadrat,percent,-391782.104376,7.313294e+06,83,yes,"{TL_WNAC5_VgComp, TL_WNAC6_VgComp}"


In [204]:
# 5) Decide whether to rebuild *all* plotVisit values or leave them alone
# rename visits if merged
if merged:
    # first assign new plotVisit index based on group
    new_visits = (
        parents['group_id'] + '_' +
        parents['year'].astype(str) + '_' +
        parents['source']
    )
    parents.index = new_visits
    parents.index.name = 'Site Code'
else:
    parents.index.name = 'Site Code'
parents

,bare ground top cover (%),bryophyte total cover (%),deciduous shrub total cover (%),deciduous tree total cover (%),evergreen shrub total cover (%),evergreen tree total cover (%),forb total cover (%),graminoid total cover (%),lichen total cover (%),litter total cover (%),...,plot_radius_m,source,year,field sampling method,cover measurement,x,y,group_id,aggregated to parent,child site codes
Site Code,,,,,,,,,,,,,,,,,,,,,
0_2018_SP,0.0,93.0,15.0,NaN,2.0,0.0,24.0,56.0,0.0,15.0,...,1.25,SP,2018,quadrat,percent,-288638.520067,7.296783e+06,0,no,{CL_EL1_Vg_Comp}
1_2018_SP,0.0,61.0,17.0,NaN,14.0,0.0,0.0,37.0,3.0,12.0,...,1.25,SP,2018,quadrat,percent,-288693.614090,7.296829e+06,1,no,{CL_EL2_Vg_Comp}
2_2018_SP,0.0,3.0,37.0,NaN,22.0,0.0,1.0,3.0,122.0,3.0,...,1.25,SP,2018,quadrat,percent,-284014.587205,7.297196e+06,2,no,{CL_PMRB1_Vg_Comp}
3_2018_SP,0.0,6.0,26.0,NaN,16.0,0.0,0.0,2.0,99.0,5.0,...,1.25,SP,2018,quadrat,percent,-283833.064238,7.297191e+06,3,no,{CL_PMRB2_Vg_Comp}
4_2018_SP,0.0,3.0,45.0,NaN,34.0,0.0,2.0,0.0,90.0,2.0,...,1.25,SP,2018,quadrat,percent,-284171.455611,7.296863e+06,4,no,{CL_PMRB3_Vg_Comp}
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
82_2017_SP,1.0,12.0,9.0,NaN,4.0,NaN,4.0,77.0,NaN,0.0,...,1.25,SP,2017,quadrat,percent,-392211.045243,7.315054e+06,82,no,{TL_WNAC3_VgComp}
69_2017_SP,0.0,37.0,23.0,NaN,23.0,NaN,0.0,19.0,NaN,0.0,...,1.25,SP,2017,quadrat,percent,-392664.527176,7.314745e+06,69,yes,"{TL_MM4_VgComp, TL_WNAC4_VgComp}"
83_2017_SP,0.0,55.0,18.0,NaN,3.0,NaN,3.0,41.0,NaN,0.0,...,1.25,SP,2017,quadrat,percent,-391782.104376,7.313294e+06,83,yes,"{TL_WNAC5_VgComp, TL_WNAC6_VgComp}"


In [205]:
# collapse duplicates by aggregating covers and taking first for metadata
cover_cols = [c for c in parents.columns if 'cover' in c and c != 'cover measurement']
info_cols = [c for c in parents.columns if c not in cover_cols]
agg_dict = {c: 'mean' for c in cover_cols}
agg_dict.update({c: 'first' for c in info_cols})
parents = parents.reset_index().groupby('Site Code', as_index=True).agg(agg_dict)

In [206]:
# drop helpers and reorder columns
parents = parents.drop(columns=['group_id','x','y'], errors=True)
cover_cols_sorted = sorted([c for c in parents.columns if 'cover' in c and c != 'cover measurement'])
info_cols_sorted = [c for c in parents.columns if c not in cover_cols_sorted]
parents = parents[cover_cols_sorted + info_cols_sorted]
parents

,bare ground top cover (%),bryophyte total cover (%),deciduous shrub total cover (%),deciduous tree total cover (%),evergreen shrub total cover (%),evergreen tree total cover (%),forb total cover (%),graminoid total cover (%),lichen total cover (%),litter total cover (%),...,water top cover (%),latitude,longitude,plot_radius_m,source,year,field sampling method,cover measurement,aggregated to parent,child site codes
Site Code,,,,,,,,,,,,,,,,,,,,,
0_2018_SP,0.0,93.0,15.0,NaN,2.0,0.0,24.0,56.0,0.0,15.0,...,0.0,64.845250,-163.782758,1.25,SP,2018,quadrat,percent,no,{CL_EL1_Vg_Comp}
10_2018_SP,0.0,12.0,107.0,NaN,0.0,0.0,51.0,38.0,1.5,70.0,...,0.0,64.859385,-163.703835,1.25,SP,2018,quadrat,percent,no,{CL_WAC4_Vg_Comp}
11_2018_SP,0.0,74.0,154.0,NaN,0.0,1.0,6.0,27.0,1.0,35.0,...,0.0,64.858285,-163.712740,1.25,SP,2018,quadrat,percent,no,{CL_WAC5_Vg_Comp}
12_2018_SP,0.0,66.0,3.0,NaN,16.0,0.0,1.0,21.0,1.0,5.0,...,0.0,64.857520,-163.693148,1.25,SP,2018,quadrat,percent,no,{CL_BEL-7-1_Vg_Comp}
13_2018_SP,0.0,28.0,1.0,NaN,6.0,0.0,2.0,31.0,2.0,8.0,...,0.0,64.860790,-163.698248,1.25,SP,2018,quadrat,percent,no,{CL_BEL2_Vg_Comp}
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
83_2017_SP,0.5,48.5,18.5,NaN,3.0,NaN,2.5,55.5,NaN,0.0,...,2.0,64.729370,-165.944615,1.25,SP,2017,quadrat,percent,yes,"{TL_WNAC5_VgComp, TL_WNAC6_VgComp}"
84_2017_SP,0.0,44.0,36.0,NaN,3.0,NaN,8.0,33.0,NaN,0.0,...,0.0,64.736497,-165.948845,1.25,SP,2017,quadrat,percent,no,{TL_WNAC7_VgComp}
8_2018_SP,0.0,28.5,54.0,NaN,8.5,0.0,25.0,34.0,17.0,19.5,...,0.0,64.860260,-163.697757,1.25,SP,2018,quadrat,percent,yes,"{CL_TT2_Vg_Comp, CL_WAC2_Vg_Comp}"


In [207]:
# clean
parents_out = parents.rename(columns=schema)
parents_out.index.name = 'Site Code'
orig_parent = pd.read_csv(f'{wdir}/{dst}_fcover_parent_01.csv')
orig_parent

,Site Code,bare ground top cover (%),bryophyte total cover (%),deciduous shrub total cover (%),deciduous tree total cover (%),evergreen shrub total cover (%),evergreen tree total cover (%),forb total cover (%),graminoid total cover (%),lichen total cover (%),...,non-vascular total cover (%),water top cover (%),latitude,longitude,plot_radius_m,source,year,field sampling method,cover measurement,aggregated to parent
0,0_2018_SP,1,9.0,15.0,0,51.0,0,0.0,3.0,48.0,...,57.0,0,64.845250,-163.782758,1.25,NGEE_SP,2018,quadrat,percent,no
1,10_2018_SP,0,53.0,60.0,0,0.0,0,8.0,44.0,0.0,...,53.0,0,64.859385,-163.703835,1.25,NGEE_SP,2018,quadrat,percent,no
2,11_2018_SP,0,30.0,70.0,0,1.0,0,2.0,63.0,0.0,...,30.0,10,64.858285,-163.712740,1.25,NGEE_SP,2018,quadrat,percent,no
3,12_2018_SP,0,3.0,37.0,0,22.0,0,1.0,3.0,122.0,...,125.0,0,64.857520,-163.693148,1.25,NGEE_SP,2018,quadrat,percent,no
4,13_2018_SP,0,6.0,26.0,0,16.0,0,0.0,2.0,99.0,...,105.0,0,64.860790,-163.698248,1.25,NGEE_SP,2018,quadrat,percent,no
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
81,83_2017_SP,0,12.0,0.0,0,0.0,0,0.0,0.0,0.0,...,12.0,0,64.743175,-165.963990,1.25,NGEE_SP,2017,quadrat,percent,no
82,84_2017_SP,0,48.5,0.0,0,0.0,0,0.0,0.0,0.0,...,48.5,0,64.729570,-165.944735,55.00,NGEE_SP,2017,quadrat,percent,yes
83,85_2017_SP,0,44.0,0.0,0,0.0,0,0.0,0.0,0.0,...,44.0,0,64.736497,-165.948845,1.25,NGEE_SP,2017,quadrat,percent,no
84,8_2018_SP,0,35.0,12.0,0,13.5,0,2.5,40.5,18.5,...,53.5,2,64.860154,-163.697310,55.00,NGEE_SP,2018,quadrat,percent,yes


In [208]:
desired_order = orig_parent['Site Code'].tolist()
parents_out = parents_out.reindex(desired_order)

# export
parents_out.to_csv(f'{wdir}/{dst}_fcover_parent_{vers}.csv')